In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from tokenization_kobert import KoBertTokenizer
import pandas as pd

In [2]:

model = BertForSequenceClassification.from_pretrained("fine_tuned_model", num_labels=3)
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

device = torch.device("cpu")
model.eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [3]:
data = pd.read_csv('all_data.csv')
data

,comment,label
0,좌배 까는건 ㅇㅂ,0
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0
4,재앙이한건햇노,2
...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1
16434,링크도 안박고 가서 글 쓰자고 선동하네,0


In [4]:
data['pred_0'] = None
data['pred_1'] = None
data['pred_2'] = None

data

,comment,label,pred_0,pred_1,pred_2
0,좌배 까는건 ㅇㅂ,0,None,None,None
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0,None,None,None
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0,None,None,None
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0,None,None,None
4,재앙이한건햇노,2,None,None,None
...,...,...,...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1,None,None,None
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2,None,None,None
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,None,None,None
16434,링크도 안박고 가서 글 쓰자고 선동하네,0,None,None,None


In [8]:
for idx, row in data.iterrows():
    sentence = row['comment']
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        #predicted_labels = torch.argmax(logits, dim=1)
    data.iat[idx,2] = logits[0][0].item()
    data.iat[idx,3] = logits[0][1].item()
    data.iat[idx,4] = logits[0][2].item()
    

In [9]:
data

,comment,label,pred_0,pred_1,pred_2
0,좌배 까는건 ㅇㅂ,0,3.143378,-1.13821,-1.954949
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0,3.143406,-1.138279,-1.955025
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0,3.14333,-1.138278,-1.954805
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0,0.944398,1.897024,-2.808636
4,재앙이한건햇노,2,0.2235,-1.735151,0.978895
...,...,...,...,...,...
16431,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1,1.05767,1.885699,-2.761267
16432,갱상도가아니라 홍어지 개좃같은 홍어년들,2,-0.731944,-0.997981,1.239424
16433,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,1.370409,1.557144,-2.824098
16434,링크도 안박고 가서 글 쓰자고 선동하네,0,3.14341,-1.138077,-1.955058


In [10]:
data.to_csv("all_data_pred.csv",encoding = 'cp949', index = False)

In [11]:
import OpenMax as om

In [12]:
ori_label = data['label']
pred_label = list(zip(data['pred_0'],data['pred_1'],data['pred_2']))
pred_label = [[a,b,c] for a,b,c in pred_label]
print(ori_label[:10])
print(pred_label[:10])

0    0
1    0
2    0
3    0
4    2
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
[[3.143378257751465, -1.1382102966308594, -1.9549494981765747], [3.1434059143066406, -1.138278841972351, -1.95502507686615], [3.143329620361328, -1.1382777690887451, -1.9548048973083496], [0.944397509098053, 1.8970237970352173, -2.808635950088501], [0.2234998196363449, -1.7351505756378174, 0.9788945317268372], [3.143378973007202, -1.1379914283752441, -1.9549669027328491], [0.8904621005058289, -1.660689115524292, 0.26887089014053345], [1.062458872795105, 1.7480236291885376, -2.827852487564087], [3.143409252166748, -1.1379376649856567, -1.9550647735595703], [3.1435089111328125, -1.137939691543579, -1.9553778171539307]]


In [13]:
openmax = om.OpenMax()
openmax.fit(pred_label,ori_label)

In [14]:
def predict_sentence(sentence):
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = [0,0,0]
    logits[0] = outputs.logits[0][0].item()
    logits[1] = outputs.logits[0][1].item()
    logits[2] = outputs.logits[0][2].item()
    
    om_result = openmax.calculate(logits)
    _class = ['혐오','성차별','일베']
    print("입력문장 : ",sentence)
    print()
    print("모델 예측 결과 - OpenMax 미적용")
    print("혐오 : ",logits[0])
    print("성차별 : ",logits[1])
    print("일베 : ",logits[2])
    print()
    print("모델 예측 결과 - OpenMax 적용")
    print("Unkown : ",om_result[0])
    print("혐오 : ",om_result[1])
    print("성차별 : ",om_result[2])
    print("일베 : ",om_result[3])
    
   

In [30]:
predict_sentence("^^ㅣ발  @ㅕ자한테 기대안함")

입력문장 :  @ㅕ자한테 기대안함

모델 예측 결과 - OpenMax 미적용
혐오 :  3.143552303314209
성차별 :  -1.13777756690979
일베 :  -1.9554868936538696

모델 예측 결과 - OpenMax 적용
Unkown :  0.0017995815520623665
혐오 :  0.9189148078389647
성차별 :  0.0396490361803668
일베 :  0.0396365744286061


In [120]:
data['label'].value_counts()

0    10431
1     4481
2     1524
Name: label, dtype: int64

In [16]:
openmax.calculate([3,3,3])

[0.9996299045134202,
 0.00012336543302618255,
 0.00012336490876910486,
 0.0001233651447845203]